In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt

reader = imageio.get_reader(r'C:\xampp\htdocs\VISUALAI\assets\video\walking people.mp4')
prev_frame = None

for frame in reader:
    gray = np.mean(frame, axis=2)
    if prev_frame is None:
        prev_frame = gray
        continue
    diff = np.abs(gray - prev_frame)
    mask = diff > 30
    plt.imshow(mask, cmap='gray')
    plt.pause(0.001)
    prev_frame = gray


In [ ]:
from io import BytesIO

import requests
import supervision as sv
from inference import get_model
from PIL import Image
from PIL.ImageFile import ImageFile


def load_image_from_url(url: str) -> ImageFile:
    response = requests.get(url)
    response.raise_for_status()  # check if the request was successful
    image = Image.open(BytesIO(response.content))
    return image


# load the image from an url
image = load_image_from_url("https://media.roboflow.com/inference/people-walking.jpg")

# load a pre-trained yolov8n model
model = get_model(model_id="yolov8n-640")

# run inference on our chosen image, image can be a url, a numpy array, a PIL image, etc.
results = model.infer(image)[0]

# load the results into the supervision Detections api
detections = sv.Detections.from_inference(results)

# create supervision annotators
bounding_box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

# annotate the image with our inference results
annotated_image = bounding_box_annotator.annotate(scene=image, detections=detections)
annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

# display the image
sv.plot_image(annotated_image)